In [ ]:
#Handwritten digit recognition using the SVM
import tkinter as tk
from tkinter import *
from tkinter import messagebox

In [ ]:
window=tk.Tk()
window.title("Handwritten digit recognition")
btnBg = "#74d4c0"
btnFg = "#d9455f"
l1=tk.Label(window,text="Handwritten Digit Recognition",bg='#def4f0',fg='#9a1f40',font=('Helvetica',20,'underline'))
l1.place(x=5,y=5)



def generate_dataset():
    import cv2
    import csv
    import glob

    header  =["label"]
    for i in range(0,784):
        header.append("pixel"+str(i))
    with open('dataset.csv', 'w') as f:
        writer = csv.writer(f)
        writer.writerow(header)

    for label in range(10):
        dirList = glob.glob("captured_images/"+str(label)+"/*.png")

        for img_path in dirList:
            im= cv2.imread(img_path)
            im_gray = cv2.cvtColor(im,cv2.COLOR_BGR2GRAY)   #convert image in grayscale form
            im_gray = cv2.GaussianBlur(im_gray,(15,15), 0)  #applies gaussian filter using 15X15 kernel size to soften the image
             #Threshold the image
            ret, im_th = cv2.threshold(im_gray,100, 255, cv2.THRESH_BINARY)
            roi= cv2.resize(im_th,(28,28), interpolation=cv2.INTER_AREA) #Shrink image to 28X28 size

            data=[]
            data.append(label)
            rows, cols = roi.shape

            ## Add pixel one by one into data array
            for i in range(rows):
                for j in range(cols):
                    k =roi[i,j]
                    if k>100:
                        k=1
                    else:
                        k=0
                    data.append(k)
            with open('dataset.csv', 'a') as f:
                writer = csv.writer(f)
                writer.writerow(data)
    messagebox.showinfo("Result","Dataset generated!!")
    
b2=tk.Button(window,text="Generate dataset from images", font=('Helvetica',15),bg= btnBg,fg= btnFg,command=generate_dataset)
b2.place(x=5, y=100)

def train_save_accuracy():
    import pandas as pd
    from sklearn.utils import shuffle
    data  =pd.read_csv('dataset.csv')
    data=shuffle(data,random_state = 0) #shuffles to prevent generalisation
    X = data.drop(["label"],axis=1)
    Y= data["label"]
    from sklearn.model_selection import train_test_split
    train_x,test_x,train_y,test_y = train_test_split(X,Y, test_size = 0.2,random_state = 0)
    import joblib
    from sklearn.svm import SVC
    classifier=SVC(kernel="linear", random_state=0)
    classifier.fit(train_x,train_y)
    joblib.dump(classifier, "model/digit_recognizer")  #saves the classifier object to use later for prediction
    from sklearn import metrics
    prediction=classifier.predict(test_x)
    acc=metrics.accuracy_score(prediction, test_y)
    messagebox.showinfo("Result",f"Accuracy is {acc*100} %")
    
b3=tk.Button(window,text="Train the model and calculate accuracy", font=('Helvetica',15),bg= btnBg,fg= btnFg,command=train_save_accuracy)
b3.place(x=5, y=150)

def prediction():
    import joblib
    import cv2
    import numpy as np
    import tkinter.filedialog as fd
    
    model=joblib.load("model/digit_recognizer")
    
    ipath = fd.askopenfilename()   #to open the filed dialog box to input image

    if len(ipath) > 0:
        im = cv2.imread(ipath)
        im_gray = cv2.cvtColor(im,cv2.COLOR_BGR2GRAY)
        im_gray  =cv2.GaussianBlur(im_gray, (15,15), 0)

        #Threshold the image
        ret, im_th = cv2.threshold(im_gray,100, 255, cv2.THRESH_BINARY)
        roi = cv2.resize(im_th, (28,28), interpolation  =cv2.INTER_AREA)

        rows,cols=roi.shape

        X = []

        ## Add pixel one by one into data array
        for i in range(rows):
            for j in range(cols):
                k = roi[i,j]
                if k>100:
                    k=1
                else:
                    k=0
                X.append(k)
        
        predictions  =model.predict([X])
        print("Prediction:",predictions[0])
        cv2.putText(im, "Prediction is: "+str(predictions[0]), (20,20), 0, 0.8,(0,255,0),2,cv2.LINE_AA)

        cv2.startWindowThread()
        cv2.namedWindow("Result")
        cv2.imshow("Result",im)
        cv2.waitKey(10000)
    cv2.destroyAllWindows()
    
b4=tk.Button(window,text="Predict", font=('Helvetica',15),bg= btnBg,fg= btnFg,command=prediction)
b4.place(x=5, y=200)

window.geometry("600x300")
window['bg'] = '#def4f0'
window.mainloop()